In [6]:
import numpy as np
# import tensorflow as tf
import tcellmatch.api as tm
from torchmetrics import Accuracy
# confusion mtx
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
ffn = tm.models.EstimatorFfn()
# data = np.load("ffn__resampled.npz")
data = np.load("ffn_data.npz")
ffn.x_train = data["x_train"]
ffn.covariates_train = data["covariates_train"]
ffn.y_train = data["y_train"]
ffn.x_test = data["x_test"]
ffn.covariates_test = data["covariates_test"]
ffn.y_test = data["y_test"]
ffn.clone_train = data["clone_train"]

In [8]:
ffn.load_idx('SAVED_IDX')

In [9]:
ffn.build_bilstm(
    topology = [10, 10],
    residual_connection=True,
    aa_embedding_dim=0,
    optimizer='adam',
    lr=0.001,
    loss='cce',
    label_smoothing=0,
)

In [10]:
# Assuming the training data is in the variables x_train_resampled, covariates_train_resampled, and y_train_resampled

# convert one-hot encoded labels to class indices
y_train_indices = np.argmax(ffn.y_train, axis=1)
classes, counts = np.unique(y_train_indices, return_counts=True)

# Identify classes with 3 or fewer samples
rare_classes = classes[counts <= 3]

# Initialize masks for x_train and covariates_train with all True values
mask_x_train = np.ones(len(y_train_indices), dtype=bool)
mask_covariates_train = np.ones(len(y_train_indices), dtype=bool)

# Set mask values for rare classes to False
for rc in rare_classes:
    mask_x_train[y_train_indices == rc] = False
    mask_covariates_train[y_train_indices == rc] = False

# Apply masks to drop rare classes
ffn.x_train = ffn.x_train[mask_x_train]
ffn.covariates_train = ffn.covariates_train[mask_covariates_train]
ffn.y_train = ffn.y_train[mask_x_train]  # Here's the corrected line

# The new training data is x_train_filtered, covariates_train_filtered, y_train_filtered

In [11]:
R = tm.utils.balance_multiclass_dataset(
        ffn.x_train,
        ffn.covariates_train,
        ffn.y_train,
        min_samples=30,
        max_samples=4000,
    )

In [16]:
ffn.x_train, ffn.covariates_train, ffn.y_train = R
ffn.idx_train_val = np.arange(0, ffn.x_train.shape[0])

In [13]:
print(ffn.x_train.shape)

(15702, 1, 40, 26)


In [15]:
print(ffn.idx_train_val.shape)

(51301,)


In [8]:
# import numpy as np
# np.savez_compressed(
#     "ffn_resampled.npz",
#     x_train=ffn.x_train,
#     covariates_train=ffn.covariates_train,
#     y_train=ffn.y_train,
#     x_test=ffn.x_test,
#     covariates_test=ffn.covariates_test,
#     y_test=ffn.y_test,
#     clone_train=ffn.clone_train
# )

In [17]:
# !! WWLO -> train esta sando renotsmarto
ffn.train(
    epochs=5,
    batch_size=8,
#   log_dir='TRAINING_RUN_1pm'
    )

IndexError: index 15702 is out of bounds for axis 0 with size 15702

In [18]:
ffn.clone_train.shape

(51343,)

In [ ]:
print(ffn.evaluate(test_only=True))
ffn.predict()

### See how balanced the ds is

In [ ]:
y_train = np.argmax(ffn.y_train, axis=1)
y_true = np.argmax(ffn.y_test, axis=1)

In [ ]:
# Count the occurrences of each value
unique_values, value_counts = np.unique(y_train, return_counts=True)

# Print the unique values and their counts
for value, count in zip(unique_values, value_counts):
    print(f"{value}: {count}")

### Evaluation

In [ ]:
# CONFUSION MATRIX
true_labels = np.argmax(ffn.y_test, axis=1)
predicted_labels = np.argmax(ffn.predictions, axis=1)

cm = confusion_matrix(true_labels, predicted_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
accuracy = Accuracy(task="multiclass", num_classes=51)
y_hat = torch.from_numpy(y_hat)
y_true = torch.from_numpy(y_true)
accuracy(y_hat, y_true)